# Imports

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import math

from sklearn.model_selection import train_test_split


from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PowerTransformer, QuantileTransformer, Normalizer

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor


!pip install --quiet optuna
import optuna


In [2]:

np.random.seed(42)


# Utils

In [3]:
#  encode a categorical column based on the mean of the target column.
def target_encode(cat_column, target_column, train , valid, test):
    mean_target = train.groupby(cat_column)[target_column].mean()
    train_encoded_series = train[cat_column].map(mean_target)
    valid_encoded_series = valid[cat_column].map(mean_target)
    test_encoded_series = test[cat_column].map(mean_target)
    return train_encoded_series , valid_encoded_series, test_encoded_series

In [4]:
# return variables that are correlated beyond thresh - drop one of the two variables
def highly_corr(df, thresh= 0.8):
  corrmat = df.corr()
  upper_tri = corrmat.where(np.triu(np.ones(corrmat.shape),k=1).astype(bool))
  to_drop = [column for column in upper_tri.columns if any(upper_tri[column] >thresh)]
  return to_drop

In [5]:
# set extreme outliers equal to a specified percentile of the data
def winsorize_data( columns, train, valid, test, lower_percentile=5, upper_percentile=95):

    for col in columns:
        lower_limit = np.percentile(train[col], lower_percentile)
        upper_limit = np.percentile(train[col], upper_percentile)
        train[col] = np.where(train[col] < lower_limit, lower_limit,
                                      np.where(train[col] > upper_limit, upper_limit, train[col]))
        valid[col] = np.where(valid[col] < lower_limit, lower_limit,
                                      np.where(valid[col] > upper_limit, upper_limit, valid[col]))
        test[col] = np.where(test[col] < lower_limit, lower_limit,
                                      np.where(test[col] > upper_limit, upper_limit, test[col]))
    return train, valid, test

In [6]:
def handle_null_values(df , mean_impute = False):
  if mean_impute:
    return df.fillna(df.mean()) #Fill missing values with column mean
  else:
    return df.dropna().reset_index() # Remove rows with null values


In [7]:
# Method - Interquartile Range (IQR)
def get_outliers(df , col , alpha = 1.5):
  Q1 = df[col].quantile(0.25)
  Q3 = df[col].quantile(0.75)
  IQR = Q3 - Q1
  lower_bound = Q1 - alpha * IQR
  upper_bound = Q3 + alpha * IQR
  outliers_iqr = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
  return len(outliers_iqr) , outliers_iqr.index

In [8]:
def info(df):
  print('-----------------------------------------------------------')
  print('Column wise null counts : ')
  null_counts = df.isnull().sum()
  print(null_counts)

  print('-----------------------------------------------------------')
  rows_with_null = df.isnull().any(axis=1).sum()
  print("Number of rows with at least one null value: ", rows_with_null)

  print('-----------------------------------------------------------')
  print('Unique Values per column : ')
  print(df.nunique())


In [9]:
def outliers_info(df, numerical):
  out = {}
  for col in numerical:
    l , _ = get_outliers(df , col )
    out[col] = l
  return out

In [10]:
def evaluation_metrics(y, predict):
  print('-----------------------------------------------------------')
  r_2 = r2_score( y, predict)
  mse = mean_squared_error(y, predict)
  rmse = math.sqrt(mse)
  print('r2 ', r_2)
  print('mse ', mse)
  print('rmse ', rmse)
  print('-----------------------------------------------------------')


# Regression

In [11]:
df = pd.read_csv('/content/HousingData.csv',  delimiter=',')

In [12]:
numerical = ['CRIM', 'ZN', 'INDUS',  'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT']
categorical = ['CHAS']
target = 'MEDV'

In [13]:
info(df)

-----------------------------------------------------------
Column wise null counts : 
CRIM       20
ZN         20
INDUS      20
CHAS       20
NOX         0
RM          0
AGE        20
DIS         0
RAD         0
TAX         0
PTRATIO     0
B           0
LSTAT      20
MEDV        0
dtype: int64
-----------------------------------------------------------
Number of rows with at least one null value:  112
-----------------------------------------------------------
Unique Values per column : 
CRIM       484
ZN          26
INDUS       76
CHAS         2
NOX         81
RM         446
AGE        348
DIS        412
RAD          9
TAX         66
PTRATIO     46
B          357
LSTAT      438
MEDV       229
dtype: int64


In [14]:
outliers_info(df, numerical + [target])

{'CRIM': 65,
 'ZN': 63,
 'INDUS': 0,
 'NOX': 0,
 'RM': 30,
 'AGE': 0,
 'DIS': 5,
 'RAD': 0,
 'TAX': 0,
 'PTRATIO': 15,
 'B': 77,
 'LSTAT': 7,
 'MEDV': 40}

In [15]:
train_temp, test = train_test_split(df,  test_size=0.2, random_state=42)
train, valid = train_test_split(train_temp, test_size=0.25, random_state=42)

del train_temp

In [16]:
train, valid, test = handle_null_values(train), handle_null_values(valid), handle_null_values(test)

In [17]:
# target encode categorical variables
for cat in categorical :
  train[cat + '_target_encode'],  valid[cat + '_target_encode'], test[cat + '_target_encode'] = target_encode( cat, target, train, valid, test)
  train.drop(cat , axis = 1, inplace = True)
  valid.drop(cat , axis = 1, inplace = True)
  test.drop(cat , axis = 1, inplace = True)
  numerical.append(cat + '_target_encode')

In [18]:
# drop highly correlated variables
to_drop = highly_corr(train[numerical])
for d in to_drop:
  train.drop(d , axis = 1, inplace = True)
  valid.drop(d , axis = 1, inplace = True)
  test.drop(d , axis = 1, inplace = True)
  numerical.remove(d)

In [19]:
train, valid, test = winsorize_data(['CRIM','ZN', 'RM' , 'B', 'MEDV'], train, valid, test)

In [20]:
X_train , y_train = train[numerical], train[target]
X_valid , y_valid = valid[numerical], valid[target]
X_test , y_test = test[numerical], test[target]

In [21]:
# todo scale data

In [22]:

scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_valid = scaler.fit_transform(X_valid)
X_test = scaler.fit_transform(X_test)

# Regression Models train

In [23]:
def objective(trial):

  regressor = trial.suggest_categorical("regressor", [# "LinearRegression", "Ridge", "Lasso"
                                                      "GradientBoostingRegressor",
                                                      "RandomForestRegressor",
                                                      "XGBRegressor"])

  if regressor == "GradientBoostingRegressor":
     params = {
              'n_estimators' : trial.suggest_int("n_estimators", 2, 20),
              'max_depth' : trial.suggest_int("max_depth", 1, 32),
              'learning_rate' : trial.suggest_float("learning_rate", 1e-7, 0.3, log=True),
              'random_state' : 42,
              'min_samples_split' : trial.suggest_int("min_samples_split", 10, 32)
              }
     clf = GradientBoostingRegressor(**params)

  elif regressor == "RandomForestRegressor":
     params = {
            "n_estimators":  trial.suggest_int("n_estimators", 2, 20),
            "max_depth":  trial.suggest_int("max_depth", 1, 32),
            # "max_features": trial.suggest_categorical( "max_features", choices=["auto", "sqrt", "log2"] ),
            "random_state": 42,
             'min_samples_split' : trial.suggest_int("min_samples_split", 10, 32)
            }
     clf = GradientBoostingRegressor(**params)

  elif regressor == "XGBRegressor":
     params = {
              'n_estimators' : trial.suggest_int("n_estimators", 2, 20),
              'max_depth' : trial.suggest_int("max_depth", 1, 32),
              'learning_rate' : trial.suggest_float("learning_rate", 1e-7, 0.3, log=True),
              'random_state' : 42
              }

     clf = XGBRegressor(**params)

  if clf == 'XGBRegressor' :
    clf.fit(X_train, y_train, early_stopping_rounds=10,
               eval_set =[X_valid, y_valid])
  else:
    clf.fit(X_train, y_train)

  predict = clf.predict(X_valid)

  r_2 = r2_score( y_valid, predict)

  mse = mean_squared_error(y_valid, predict)
  rmse = math.sqrt(mse)

  return rmse

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100,  timeout=600)

print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2024-04-10 22:08:58,264] A new study created in memory with name: no-name-2fd0990e-ac2f-4bd8-af6f-d1b2c63c72a9
[I 2024-04-10 22:08:58,442] Trial 0 finished with value: 3.022606805315279 and parameters: {'regressor': 'RandomForestRegressor', 'n_estimators': 19, 'max_depth': 27, 'min_samples_split': 29}. Best is trial 0 with value: 3.022606805315279.
[I 2024-04-10 22:08:59,030] Trial 1 finished with value: 7.521926751516057 and parameters: {'regressor': 'XGBRegressor', 'n_estimators': 18, 'max_depth': 14, 'learning_rate': 2.726466904408726e-05}. Best is trial 0 with value: 3.022606805315279.
[I 2024-04-10 22:08:59,072] Trial 2 finished with value: 2.7213099962514287 and parameters: {'regressor': 'GradientBoostingRegressor', 'n_estimators': 20, 'max_depth': 8, 'learning_rate': 0.2122614196838708, 'min_samples_split': 32}. Best is trial 2 with value: 2.7213099962514287.
[I 2024-04-10 22:08:59,089] Trial 3 finished with value: 4.600346849230843 and parameters: {'regressor': 'RandomForest

Number of finished trials: 100
Best trial:
  Value: 2.526600501104273
  Params: 
    regressor: GradientBoostingRegressor
    n_estimators: 20
    max_depth: 10
    learning_rate: 0.2966738552456868
    min_samples_split: 32


In [24]:
name  = trial.params['regressor']
trial.params.pop('regressor')
params = trial.params

if name == "GradientBoostingRegressor":
    model = GradientBoostingRegressor(**params)
elif name == "RandomForestRegressor":
    model = GradientBoostingRegressor(**params)
elif name == "XGBRegressor":
    model = XGBRegressor(**params)

In [25]:

model.fit(X_train , y_train)

print('Train')
predict = model.predict(X_train)
evaluation_metrics(y_train, predict )

print('Valid')
predict = model.predict(X_valid)
evaluation_metrics(y_valid, predict )

print('Test')
predict = model.predict(X_test)
evaluation_metrics(y_test, predict )

Train
-----------------------------------------------------------
r2  0.9958805648084803
mse  0.3017621393132329
rmse  0.5493288080132271
-----------------------------------------------------------
Valid
-----------------------------------------------------------
r2  0.8868358629670514
mse  6.383710092180364
rmse  2.526600501104273
-----------------------------------------------------------
Test
-----------------------------------------------------------
r2  0.9115634456729633
mse  5.434962727491185
rmse  2.331300651458577
-----------------------------------------------------------


In [26]:
from optuna.visualization import plot_optimization_history

plotly_config = {"staticPlot": True}
fig = plot_optimization_history(study)
# fig.show()
fig.show(config=plotly_config)

In [27]:
# optuna.visualization.plot_slice(study)

In [28]:
# optuna.visualization.plot_contour(study, params=["n_estimators", "max_depth"])

In [29]:
# https://www.kaggle.com/code/bextuychiev/no-bs-guide-to-hyperparameter-tuning-with-optuna#Defining-the-search-space

In [30]:
# todo write cross val for smaller dataset